In [ ]:
import pandas as pd
import os 

In [ ]:
os.chdir('C:\\Users\\Adnan Shahbaaz\\Desktop\\Big Derby 22')

In [ ]:
# read data
df = pd.read_csv('Data/nyra_race_table.csv')

In [ ]:
dates = df.race_date 

# // Code venues as id's on trakus website

df.loc[df['track_id']== 'AQU',"wtrackid"] = '24'
df.loc[df['track_id']== 'BEL',"wtrackid"] = '23'
df.loc[df['track_id']== 'SAR',"wtrackid"] = '25'

df.track_id.unique() #check

venues = df.wtrackid 

In [ ]:
# // unique urls with date and venue in dataset

urls_a = 'https://tnetwork.trakus.com/tnet/t_NYRA.aspx?Venue=' + venues + '&Type=TBRED&Date=' + dates

urls = urls_a.unique()

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import time

In [ ]:
#// Get relative url's for different races on each raceday at relevant venue 

import cchardet # for faster parsing
import lxml

rlink = []

start = time.time()

r = requests.Session()

for url in urls:
    req = r.get(url)
    soup  = bs(req.text,"lxml")
    racelist = soup.find_all('table',attrs={"class":"DailyRaceList"})
   
    for tags in racelist:
        atag = tags.find_all('a')
    
    for reflink in atag:    
        href =  reflink.get('href') 
        rlink.append('https://tnetwork.trakus.com/tnet/' + href)

end = time.time()

In [ ]:
rlink[:10]

In [ ]:
len(rlink) - len(urls_a) # excess of 31 urls found to unique races

In [ ]:
end - start # execution time 4m!! :(

In [ ]:
# collect soups

start = time.time()

soups = []

r = requests.Session()

for url in rlink:
    req = r.get(url)
    soups.append(bs(req.text,"lxml"))

end = time.time()

In [ ]:
end - start # 14m :((

In [ ]:
# scrape relative urls for horse numbers 

import re

horsenumbers = []
blank = []

start = time.time()

for soup in soups:
    images = soup.find_all('img')
    horse_n = [] # clear horse_n after outer loop
    
    for image in images: 
        gifsrc = image['src']
        r = re.compile(r'(\d+).gif') # .gif to ensure pulling only number cards
        s = r.findall(gifsrc)
        if  s != blank:
            horse_n += s
    
    horsenumbers.append(horse_n)

end = time.time()


In [ ]:
# write data

rlink_df = pd.DataFrame(rlink)

horsenumbers_df = pd.DataFrame(horsenumbers)
horsenumbers_df = horsenumbers_df.rename(columns= {0:'hn_1',1:'hn_2',2:'hn_3',3:'hn_4',4:'hn_5',5:'hn_6',6:'hn_7'})

j = rlink_df.join(horsenumbers_df) # join urls with horse numbers 

j = j.rename(columns={0:'race_link'}) 
j.to_csv('reflink+horsenumber.csv',index=False)

In [ ]:
len(j[j['hn_1'].isnull()]) # no data for 40 races??